![Credit card being held in hand](credit_card.jpg)

Commercial banks receive _a lot_ of applications for credit cards. Many of them get rejected for many reasons, like high loan balances, low income levels, or too many inquiries on an individual's credit report, for example. Manually analyzing these applications is mundane, error-prone, and time-consuming (and time is money!). Luckily, this task can be automated with the power of machine learning and pretty much every commercial bank does so nowadays. In this workbook, you will build an automatic credit card approval predictor using machine learning techniques, just like real banks do.

### The Data

The data is a small subset of the Credit Card Approval dataset from the UCI Machine Learning Repository showing the credit card applications a bank receives. This dataset has been loaded as a `pandas` DataFrame called `cc_apps`. The last column in the dataset is the target value.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# Load the dataset
cc_apps = pd.read_csv("cc_approvals.data", header=None)
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+


In [2]:
cc_apps

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,g,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,g,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,g,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,g,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,s,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,g,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,g,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,g,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,g,750,-


In [ ]:
cc_apps.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], dtype='int64')

In [ ]:
cc_apps.isnull()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,False,False,False,False,False,False,False,False,False,False,False,False,False,False
686,False,False,False,False,False,False,False,False,False,False,False,False,False,False
687,False,False,False,False,False,False,False,False,False,False,False,False,False,False
688,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
# cc_apps.fillna(0, inplace=True)

In [4]:
cc_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    int64  
 13  13      690 non-null    object 
dtypes: float64(2), int64(2), object(10)
memory usage: 75.6+ KB


In [20]:
cc_apps[[13]].value_counts()

13
-     383
+     307
Name: count, dtype: int64

In [7]:
cc_apps.columns

Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], dtype='int64')

In [10]:
df = pd.get_dummies(cc_apps, columns=[0, 1, 3, 4, 5, 6, 8, 9, 11, 13], drop_first=True)

In [11]:
df

,2,7,10,12,0_a,0_b,1_15.17,1_15.75,1_15.83,1_15.92,...,6_j,6_n,6_o,6_v,6_z,8_t,9_t,11_p,11_s,13_-
0,0.000,1.25,1,0,False,True,False,False,False,False,...,False,False,False,True,False,True,True,False,False,False
1,4.460,3.04,6,560,True,False,False,False,False,False,...,False,False,False,False,False,True,True,False,False,False
2,0.500,1.50,0,824,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
3,1.540,3.75,5,3,False,True,False,False,False,False,...,False,False,False,True,False,True,True,False,False,False
4,5.625,1.71,0,0,False,True,False,False,False,False,...,False,False,False,True,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,10.085,1.25,0,0,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
686,0.750,2.00,2,394,True,False,False,False,False,False,...,False,False,False,True,False,False,True,False,False,True
687,13.500,2.00,1,1,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
688,0.205,0.04,0,750,False,True,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True


In [13]:
df.columns

Index([        2,         7,        10,        12,     '0_a',     '0_b',
       '1_15.17', '1_15.75', '1_15.83', '1_15.92',
       ...
           '6_j',     '6_n',     '6_o',     '6_v',     '6_z',     '8_t',
           '9_t',    '11_p',    '11_s',    '13_-'],
      dtype='object', length=389)

df_encoded

In [18]:
df[df.columns[-1]].value_counts()

13_-
True     383
False    307
Name: count, dtype: int64

In [23]:
df.columns = df.columns.astype(str)

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df.iloc[:, -1], test_size=0.2, random_state=1)

In [25]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [28]:
X_train

array([[ 0.39794576,  0.53803848,  0.93664093, ...,  1.18763646,
        -0.12126781, -0.29430286],
       [-0.63192878, -0.4664883 , -0.46723051, ..., -0.84200851,
        -0.12126781,  3.39786029],
       [-0.68431304, -0.60179791, -0.46723051, ..., -0.84200851,
        -0.12126781, -0.29430286],
       ...,
       [ 0.0836402 ,  3.33247608, -0.46723051, ..., -0.84200851,
        -0.12126781, -0.29430286],
       [-0.57954452, -0.02526131,  0.53553481, ...,  1.18763646,
        -0.12126781, -0.29430286],
       [ 1.49801521, -0.49149116, -0.06612438, ...,  1.18763646,
        -0.12126781, -0.29430286]])

In [33]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [34]:
y_pred = lr.predict(X_test)


In [35]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy:", accuracy)

Test accuracy: 0.8478260869565217


In [36]:
confusion_matrix(y_test, y_pred)

array([[51, 12],
       [ 9, 66]])

In [41]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(lr, X_train, y_train, cv=5, scoring='accuracy')
print("Cross-validation scores:", scores)

Cross-validation scores: [0.83783784 0.83783784 0.71818182 0.80909091 0.81818182]


In [42]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear', 'saga'],
    'max_iter': [100, 500, 1000,5000]
    }


In [48]:
gd = GridSearchCV(estimator = lr,
                  param_grid=param_grid,
                  verbose = 2,
                  cv=5)


In [49]:
gd.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV] END ................C=0.001, max_iter=100, solver=lbfgs; total time=   0.0s
[CV] END ................C=0.001, max_iter=100, solver=lbfgs; total time=   0.0s
[CV] END ................C=0.001, max_iter=100, solver=lbfgs; total time=   0.0s
[CV] END ................C=0.001, max_iter=100, solver=lbfgs; total time=   0.0s
[CV] END ................C=0.001, max_iter=100, solver=lbfgs; total time=   0.0s
[CV] END ............C=0.001, max_iter=100, solver=liblinear; total time=   0.0s
[CV] END ............C=0.001, max_iter=100, solver=liblinear; total time=   0.0s
[CV] END ............C=0.001, max_iter=100, solver=liblinear; total time=   0.0s
[CV] END ............C=0.001, max_iter=100, solver=liblinear; total time=   0.0s
[CV] END ............C=0.001, max_iter=100, solver=liblinear; total time=   0.0s
[CV] END .................C=0.001, max_iter=100, solver=saga; total time=   0.2s
[CV] END .................C=0.001, max_iter=100

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END ...................C=0.1, max_iter=100, solver=saga; total time=   0.5s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END ...................C=0.1, max_iter=100, solver=saga; total time=   0.4s


...

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END ..................C=100, max_iter=1000, solver=saga; total time=   2.5s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END ..................C=100, max_iter=1000, solver=saga; total time=   2.4s
[CV] END .................C=100, max_iter=5000, solver=lbfgs; total time=   0.1s
[CV] END .................C=100, max_iter=5000, solver=lbfgs; total time=   0.2s
[CV] END .................C=100, max_iter=5000, solver=lbfgs; total time=   0.1s
[CV] END .................C=100, max_iter=5000, solver=lbfgs; total time=   0.2s
[CV] END .................C=100, max_iter=5000, solver=lbfgs; total time=   0.4s
[CV] END .............C=100, max_iter=5000, solver=liblinear; total time=   0.2s
[CV] END .............C=100, max_iter=5000, solver=liblinear; total time=   0.2s
[CV] END .............C=100, max_iter=5000, solver=liblinear; total time=   0.2s
[CV] END .............C=100, max_iter=5000, solver=liblinear; total time=   0.2s
[CV] END .............C=100, max_iter=5000, solver=liblinear; total time=   0.2s
[CV] END ..................C=100, max_iter=5000, solver=saga; total time=   8.7s
[CV] END ..................C

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'max_iter': [100, 500, 1000, 5000],
                         'solver': ['lbfgs', 'liblinear', 'saga']},
             verbose=2)

In [50]:
print(gd.best_params_)

{'C': 0.001, 'max_iter': 100, 'solver': 'liblinear'}


In [53]:
results = pd.DataFrame(gd.cv_results_)
results[['mean_test_score', 'rank_test_score']]

,mean_test_score,rank_test_score
0,0.764488,68
1,0.840639,1
2,0.768092,65
3,0.764488,68
4,0.840639,1
...,...,...
67,0.787961,49
68,0.798788,45
69,0.780737,57
70,0.787961,49


In [54]:
print(gd.best_score_)

0.8406388206388208


In [55]:
lr2= LogisticRegression(C=0.001, max_iter=100, solver='liblinear')
lr2.fit(X_train, y_train)
y_pred2 = lr2.predict(X_test)
accuracy2 = accuracy_score(y_test, y_pred2)
print("Test accuracy:", accuracy2)

Test accuracy: 0.8623188405797102


Test accuracy with best parameters from GridSearchCV: 0.8623188405797102


In [63]:
row = ['b',  26.00,  7.811,  'y',  'gg',  'ff',  'ff',   9.36 , 't', 't',   3,  'p',  842  ]

row_cat = pd.get_dummies(row)
row_cat

,3,7.811,9.36,26.0,842,b,ff,gg,p,t,y
0,False,False,False,False,False,True,False,False,False,False,False
1,False,False,False,True,False,False,False,False,False,False,False
2,False,True,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,True,False,False,False
5,False,False,False,False,False,False,True,False,False,False,False
6,False,False,False,False,False,False,True,False,False,False,False
7,False,False,True,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,True,False
9,False,False,False,False,False,False,False,False,False,True,False
